Test file created to test model and pipeline scripts

In [1]:
import pandas as pd
import numpy as np

In [2]:
df27 = pd.read_parquet('part27.snappy.parquet')
df51 = pd.read_parquet('part51.snappy.parquet')
df71 = pd.read_parquet('part71.snappy.parquet')

In [3]:
df27.info()
df51.info()
df71.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14419 entries, 0 to 14418
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   pdv            14419 non-null  object
 1   premise        14419 non-null  object
 2   categoria_pdv  14419 non-null  object
 3   zipcode        14419 non-null  int32 
dtypes: int32(1), object(3)
memory usage: 394.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6560698 entries, 0 to 6560697
Data columns (total 11 columns):
 #   Column               Dtype  
---  ------               -----  
 0   internal_store_id    object 
 1   internal_product_id  object 
 2   distributor_id       object 
 3   transaction_date     object 
 4   reference_date       object 
 5   quantity             float64
 6   gross_value          float64
 7   net_value            float64
 8   gross_profit         float64
 9   discount             float64
 10  taxes                float64
dtypes: float64(6), object(

In [4]:
df_jan22 = df51[
    (pd.to_datetime(df51["transaction_date"]).dt.year == 2022) &
    (pd.to_datetime(df51["transaction_date"]).dt.month == 1)
].copy()

df_jan23 = df_jan22.copy()
df_jan23["transaction_date"] = df_jan23["transaction_date"] + pd.DateOffset(years=1)
df_jan23["reference_date"] = df_jan23["reference_date"] + pd.DateOffset(years=1)

In [5]:
# shift dates to january 2023
df_jan22["transaction_date"] = pd.to_datetime(df_jan22["transaction_date"])
df_jan22["reference_date"] = pd.to_datetime(df_jan22["reference_date"])

In [6]:
rng = np.random.default_rng(42)
df_jan23["quantity"] = (
    df_jan23["quantity"] * (1 + rng.uniform(-0.2, 0.2, size=len(df_jan23)))
).round(0).clip(lower=0)

In [7]:
df_jan23 = df_jan23.reset_index(drop=True)

df_jan23.head()

,internal_store_id,internal_product_id,distributor_id,transaction_date,reference_date,quantity,gross_value,net_value,gross_profit,discount,taxes
0,8277069645128751773,2412225681255611361,7,2023-01-12 00:00:00,2023-01-01 00:00:00,3.0,240.809998,240.429596,28.299597,0.000000,0.380409
1,7794032741516778774,3726736891643803768,4,2023-01-27 00:00:00,2023-01-01 00:00:00,1.0,20.500000,20.500000,6.000000,0.000000,0.000000
2,7731233724512594516,500478784353013717,5,2023-01-11 00:00:00,2023-01-01 00:00:00,8.0,170.309998,168.735001,32.284000,38.220001,1.575000
3,8737393492013252421,4451855961536069898,5,2023-01-10 00:00:00,2023-01-01 00:00:00,1.0,30.750000,30.515625,8.626625,0.000000,0.234375
4,3444141618594117443,3262679882836704514,7,2023-01-14 00:00:00,2023-01-01 00:00:00,3.0,66.300003,65.760002,18.960003,0.000000,0.540000


In [8]:
df27_filt = df27[df27["pdv"].isin(df_jan23["internal_store_id"].unique())].copy()

df71_filt = df71[df71["produto"].isin(df_jan23["internal_product_id"].unique())].copy()

In [9]:
# reset indexes
df27_filt = df27_filt.reset_index(drop=True)
df71_filt = df71_filt.reset_index(drop=True)

print("df27 filtered:", df27_filt.shape)
print("df71 filtered:", df71_filt.shape)

df27 filtered: (9362, 4)
df71 filtered: (2933, 8)


In [10]:
df27_filt.to_parquet("part27_jan.parquet", engine="pyarrow", index=False)
df71_filt.to_parquet("part71_jan.parquet", engine="pyarrow", index=False)
df_jan23.to_parquet("part51_jan.parquet", engine="pyarrow", index=False)